In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.vision import *

In [ ]:
path="../input/aptos2019-blindness-detection/"

In [ ]:
!ls ../input/resnet50/

In [ ]:
# copy pretrained weights for resnet50 to the folder fastai will search by default
Path('/tmp/.cache/torch/checkpoints/').mkdir(exist_ok=True, parents=True)
!cp '../input/resnet50/resnet50.pth' '/tmp/.cache/torch/checkpoints/resnet50-19c8e357.pth'

In [ ]:
%%time
train_df = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')
test_df = pd.read_csv('../input/aptos2019-blindness-detection/test.csv')
train_df.head()
test_df.head()

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
train_img=open_image('../input/aptos2019-blindness-detection/train_images/002c21358ce6.png')
#train_img

In [ ]:
train_img.size

In [ ]:
test_img=open_image('../input/aptos2019-blindness-detection/test_images/10407824638c.png')
#test_img

In [ ]:
test_img.size

In [ ]:
train_df['diagnosis'].hist(figsize = (10, 5))

In [ ]:
len(train_df.index)

In [ ]:
#!ls ../input/aptos2019-blindness-detection/train_images

In [ ]:
#tfms = get_transforms(flip_vert=True, max_lighting=0.1, max_zoom=1.05, max_warp=0.)
tfms = get_transforms(do_flip=True,flip_vert=True,max_rotate=360,max_warp=0,max_zoom=1.2,max_lighting=0.1,p_lighting=0.5) 

In [ ]:
bs = 64 #smaller batch size is better for training, but may take longer
sz=224

In [ ]:
np.random.seed(42)
src = (ImageList.from_csv(path, 'train.csv', folder='train_images', suffix='.png')
       .split_by_rand_pct(0.2)
       .label_from_df())

In [ ]:
data= (src.transform(tfms, size=sz, padding_mode='zeros') #Data augmentation
            .databunch(bs=bs, num_workers=2) #DataBunch
            .normalize(imagenet_stats) #Normalize     
           )

In [ ]:
print(data.classes);data.c

In [ ]:
data.show_batch(rows=6, figsize=(12,9))

In [ ]:
arch = models.resnet50

In [ ]:
MODEL_PATH = "../model/"

In [ ]:
learn = cnn_learner(data, arch, metrics=[error_rate,accuracy,KappaScore(weights="quadratic")],model_dir=MODEL_PATH)

In [ ]:
#learn.freeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot(suggestion=True)

In [ ]:
lr=3e-3

In [ ]:
data.train_ds[0][0].shape

In [ ]:
learn.fit_one_cycle(10,max_lr = lr)
#learn.fit_one_cycle(5, slice(lr))

In [ ]:
learn.save('stage-1-rn50')

In [ ]:
learn.load('stage-1-rn50')

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(8, max_lr = slice(2e-5,lr/5))

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.save('stage-2-rn50')

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
interp.plot_confusion_matrix()

In [ ]:
learn.recorder.plot_lr(show_moms=True)


In [ ]:
sample_df = pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')
sample_df.head()

In [ ]:
learn.data.add_test(ImageList.from_df(
    sample_df, '../input/aptos2019-blindness-detection/',
    folder='test_images',
    suffix='.png'
))

In [ ]:
preds,y = learn.TTA(ds_type=DatasetType.Test)

In [ ]:
sample_df.diagnosis = preds.argmax(1)
sample_df.head()

In [ ]:
sample_df.to_csv('submission.csv',index=False)
_ = sample_df.hist()